In [1]:
import pandas as pd

popden = pd.read_csv("population-density.xls.csv",index_col = "Neighborhood")
popden['Pop. 2010'] = popden['Pop. 2010'].astype(int)
popden['Land Area (acres)'] = popden['Land Area (acres)'].astype(float)
rpopden = popden.iloc[:,[8,16,19]]

rpopden.sort_values(by = ['Pop. 2010','Land Area (acres)'], ascending = False)

,Pop. 2010,Land Area (acres),Persons / acre (2010)
Neighborhood,,,
Squirrel Hill South,15110,1709.44,8.8
Shadyside,13915,589.44,23.6
Brookline,13214,1332.48,9.9
Squirrel Hill North,11363,782.08,14.5
North Oakland,10551,319.36,33.0
Carrick,10113,1070.72,9.4
Mount Washington,8799,729.60,12.1
Bloomfield,8442,449.28,18.8
Beechview,7974,934.40,8.5
